In [2]:
import tensorflow as tf

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)
assert "2." in tf.__version__, "This notebook is designed to run on TensorFlow 2.x"


TensorFlow version: 2.14.0


In [8]:
# Additional imports here
import os
from tensorflow.keras import layers, regularizers

In [1]:
# Mount drive for data access
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Initialize directories
gan_data_dir = '/content/drive/My Drive/ML-GroupProject/Processed_Data/GAN_Training'

classifier_train_dir = '/content/drive/My Drive/ML-GroupProject/Processed_Data/Classifier_Training'

classifier_test_dir = '/content/drive/My Drive/ML-GroupProject/Processed_Data/Classifier_Testing'

In [7]:
# Load dataset for GAN Training

batch_size = 32  # Adjust as needed

# Function to normalize images
def normalize_image(image):
    # Normalize the images to [-1, 1]
    image = (image - 127.5) / 127.5
    return image

# Load the dataset
gan_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    gan_data_dir,
    label_mode=None,  # No labels needed as this is unsupervised learning
    image_size=(256, 256),  # Images are already of this size
    batch_size=batch_size,
    shuffle=True
).map(normalize_image).cache().prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

Found 3464 files belonging to 1 classes.


In [ ]:
# Define functions for layer definitions
# This will help with testing different achitectures

# Regularization weight-    *might need to tune this value
l2_weight = 0.000025

# Dense layer
def dense(inputs, units):
    return layers.Dense(
        units, activation=None,
        kernel_initializer='glorot_uniform',
        kernel_regularizer=regularizers.l2(l2_weight),
        bias_regularizer=regularizers.l2(l2_weight))(inputs)

# Batch normalization layer
def batch_norm(inputs, is_training=True):
    return layers.BatchNormalization(momentum=0.999, epsilon=0.001)(inputs, training=is_training)

# Convolutional layer
def conv2d(inputs, filters, kernel_size, stride):
    return layers.Conv2D(
        filters, kernel_size, strides=stride, activation=None, padding='same',
        kernel_initializer='glorot_uniform',
        kernel_regularizer=regularizers.l2(l2_weight),
        bias_regularizer=regularizers.l2(l2_weight))(inputs)

# Transposed convolutional layer (deconvolutional layer)
def deconv2d(inputs, filters, kernel_size, stride):
    return layers.Conv2DTranspose(
        filters, kernel_size, strides=stride, activation='relu', padding='same',
        kernel_initializer='glorot_uniform',
        kernel_regularizer=regularizers.l2(l2_weight),
        bias_regularizer=regularizers.l2(l2_weight))(inputs)

In [ ]:
# Define Generator architecture

def make_generator_model(noise_dim, is_training=True, l2_weight):
    # Input: Noise vector
    noise = layers.Input(shape=(noise_dim,))

    # First dense layer
    net = dense(noise, 8*8*256)                 # 8*8 represents spatial dims of feature map, 256 number of channels (depth) of feature map
    net = batch_norm(net, is_training)
    net = layers.LeakyReLU()(net)

    # Reshape to start the convolutional stack
    net = layers.Reshape((8, 8, 256))(net)

    # Deconvolutional layers: stride (1,1) so spatial dims is 8x8, 128 is the number of filters (depth)
    net = deconv2d(net, 128, (5, 5), (1, 1), weight_decay)
    net = batch_norm(net, is_training)
    net = layers.LeakyReLU()(net)

    # Upscale to 16x16:   Stride (2,2)
    net = deconv2d(net, 64, (5, 5), (2, 2), weight_decay)
    net = batch_norm(net, is_training)
    net = layers.LeakyReLU()(net)

    # Upscale to 32x32:   Stride (2,2)
    net = deconv2d(net, 32, (5,5), (2,2), weight_decay)
    net = batch_norm(net, is_training)
    net = layers.LeakyReLU()(net)

    # Upscale to 64x64:   Stride (2,2)
    net = deconv2d(net, 16, (5,5), (2,2), weight_decay)
    net = batch_norm(net, is_training)
    net = layers.LeakyReLU()(net)

    # Final deconvolutional layer: Upscale to 256x256: Stride (4,4) with 1 filter for greyscale output (if RGB, change 1 to 3)
    net = deconv2d(net, 1, (5, 5), (4, 4), weight_decay)
    net = layers.Activation('tanh')(net)  # Outputs in [-1, 1]

    # Create the Keras model
    return tf.keras.Model(inputs=noise, outputs=net)


In [ ]:
# Define Discriminator architecture   ##### STILL WORKING ON THIS!!!  #####

def make_discriminator_model(is_training=True, l2_weight):
    # Input: Image
    input_image = layers.Input(shape=(256, 256, 1))  # Update the shape for grayscale images

    # Convolutional layers: stride (2,2) to downsample the image
    net = conv2d(input_image, 64, (5, 5), (2, 2), l2_weight)
    net = layers.LeakyReLU()(net)
    net = layers.Dropout(0.3)(net, training=is_training)

    net = conv2d(net, 128, (5, 5), (2, 2), l2_weight)
    net = layers.LeakyReLU()(net)
    net = layers.Dropout(0.3)(net, training=is_training)

    # Continue adding more layers if needed...

    # Flatten and final dense layer
    net = layers.Flatten()(net)
    net = dense(net, 1)  # Output layer with a single neuron for binary classification

    # Create the Keras model
    return tf.keras.Model(inputs=input_image, outputs=net)
